In [1]:
"""
LICENSE MIT
2021
Guillaume Rozier
Website : http://www.covidtracker.fr
Mail : guillaume.rozier@telecomnancy.net

README:
This file contains scripts that download data from data.gouv.fr and then process it to build many graphes.
I'm currently cleaning the code, please ask me if something is not clear enough.

The charts are exported to 'charts/images/france'.
Data is download to/imported from 'data/france'.
Requirements: please see the imports below (use pip3 to install them).

"""

"\nLICENSE MIT\n2021\nGuillaume Rozier\nWebsite : http://www.covidtracker.fr\nMail : guillaume.rozier@telecomnancy.net\n\nREADME:\nThis file contains scripts that download data from data.gouv.fr and then process it to build many graphes.\nI'm currently cleaning the code, please ask me if something is not clear enough.\n\nThe charts are exported to 'charts/images/france'.\nData is download to/imported from 'data/france'.\nRequirements: please see the imports below (use pip3 to install them).\n\n"

In [2]:
import pandas as pd
import json
import france_data_management as data
import math
import numpy as np

show_charts = False
PATH_STATS = "../../data/france/stats/"
PATH = "../../"

In [3]:
df_regions_meta = pd.read_csv(PATH+"data/france/population_grandes_regions.csv")

In [4]:
data.download_data_obepine()
df_obepine = data.import_data_obepine()
df_obepine_france = df_obepine.groupby("Date").mean().reset_index()

/Users/guillaumerozier/Documents/Education/Covid-19_new/covid-19-org/src/france/france_data_management.py:119: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(PATH + 'data/france/donnees_obepine_regions.csv', sep=None)


In [5]:
df_adm_hosp_clage = data.import_data_hosp_ad_age()
df_adm_hosp_clage["jour"] = pd.to_datetime(df_adm_hosp_clage.Semaine+"-6", format="%Y-S%U-%w").fillna(0)
df_adm_hosp_clage_france = df_adm_hosp_clage.groupby(["jour", "cl_age90"]).sum().fillna(0).reset_index()

In [6]:
data.download_data()

36it [00:10,  2.30it/s]                      

In [7]:
df, df_confirmed, dates, df_new, df_tests, df_deconf, df_sursaud, df_incid, df_tests_viros = data.import_data()


  0%|          | 0/8 [00:00<?, ?it/s]/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):

 75%|███████▌  | 6/8 [00:07<00:02,  1.48s/it]
21it [00:15,  1.20s/it]                      
28it [05:03, 13.18s/it]
36it [05:04,  9.25s/it]

In [8]:
df_new_france = data.import_data_new().groupby("jour").sum().reset_index()

In [9]:
df_vue_ensemble = data.import_data_vue_ensemble()

In [10]:
df_education = data.import_data_education()

df_niveaux_scolaires_fra = data.download_and_import_data_niveaux_scolaires_fra()
df_niveaux_scolaires_reg = data.download_and_import_data_niveaux_scolaires_reg()
df_niveaux_scolaires_dep = data.download_and_import_data_niveaux_scolaires_dep()

In [11]:
#df_vacsi_a = data.import_data_vacsi_a_fra()
#df_vacsi_a_reg = data.import_data_vacsi_a_reg()
#df_vacsi_a_dep = data.import_data_vacsi_a_dep()

df_vacsi = data.import_data_vacsi_fra() #df_vacsi_a.groupby("jour").sum().reset_index()
df_vacsi_reg = data.import_data_vacsi_reg() #df_vacsi_a_reg.groupby(["jour", "reg"]).sum().reset_index()
df_vacsi_reg = df_vacsi_reg.merge(df_regions_meta, left_on="reg", right_on="code").rename({"n_tot_dose1": "n_cum_dose1"}, axis=1)

df_vacsi_dep = data.import_data_vacsi_dep().rename({"n_tot_dose1": "n_cum_dose1"}, axis=1)
#df_vacsi_a_dep.groupby(["jour", "dep"]).sum().reset_index().rename({"n_tot_dose1": "n_cum_dose1"}, axis=1)

In [12]:
df_metro = data.import_data_metropoles()
df_metro["jour"] = df_metro["semaine_glissante"].map(lambda x: x[11:])

df_metro_65 = df_metro[df_metro["clage_65"] == 65]
df_metro_0 = df_metro[df_metro["clage_65"] == 0]
metropoles = list(dict.fromkeys(list(df_metro['Metropole'].dropna().values))) 

In [13]:
df_tests_viros_enrichi = data.import_data_tests_viros()
df_tests_viros_enrichi = df_tests_viros_enrichi.drop("regionName_y", axis=1).rename({"regionName_x": "regionName"}, axis=1)

In [14]:
df_incid_clage = df_incid.copy()

df_incid_fra_clage = data.import_data_tests_sexe()
df_incid_fra = df_incid_fra_clage[df_incid_fra_clage["cl_age90"]==0]
df_france = df.groupby(["jour"]).sum().reset_index()
df_incid = df_incid[df_incid.cl_age90 == 0]

df_sursaud_france = df_sursaud.groupby(["date_de_passage"]).sum().reset_index()
df_sursaud_regions = df_sursaud.groupby(["date_de_passage", "regionName"]).sum().reset_index()

#df_new_france = df_new.groupby(["jour"]).sum().reset_index()
df_new_regions = df_new.groupby(["jour", "regionName"]).sum().reset_index()

In [15]:
df_incid_clage_regions = df_incid_clage.groupby(["regionName", "jour", "cl_age90"]).sum().reset_index()

In [16]:
df_tests_viros_regions = df_tests_viros_enrichi.groupby(["regionName", "jour", "cl_age90"]).sum().reset_index()
df_tests_viros_france = df_tests_viros_enrichi.groupby(["jour", "cl_age90"]).sum().reset_index()

In [17]:
df_hosp_clage = data.import_data_hosp_clage()
df_hosp_clage_france = df_hosp_clage.groupby(["jour", "cl_age90"]).sum().reset_index()
df_hosp_clage_regions = df_hosp_clage.groupby(["regionName", "jour", "cl_age90"]).sum().reset_index()

In [18]:
departements = list(dict.fromkeys(list(df_incid['dep'].values)))
regions = list(dict.fromkeys(list(df_incid['regionName'].dropna().values))) 
clage_list = list(dict.fromkeys(list(df_incid_fra_clage['cl_age90'].dropna().values))) 

df_regions = df.groupby(["jour", "regionName"]).sum().reset_index()
df_incid_regions = df_incid.groupby(["jour", "regionName"]).sum().reset_index()


zone_a = ["zone_a", "01", "03", "07", "15", "16", "17", "19", "21", "23", "24", "25", "26", "33", "38", "39", "40", "42", "43", "47", "58", "63", "64", "69", "70", "71", "73", "74", "79", "86", "90"]
zone_b = ["zone_b", "02", "04", "05", "06", "08", "10", "13", "14", "18", "22", "27", "28", "29", "35", "36", "37", "41", "44", "45", "49", "50", "51", "52", "53", "54", "55", "56", "57", "59", "60", "61", "62", "67", "68", "72", "76", "80", "83", "84", "85", "88"]
zone_c = ["zone_c", "09", "11", "12", "30", "31", "32", "34", "46", "48", "65", "66", "75", "77", "78", "81", "82", "91", "92", "93", "94", "95"]

confines_mars_2021 = ["confines_mars_2021", "02", "06", "27", "59", "60", "62", "75", "76", "77", "78", "80", "91", "92", "93", "94", "95"]

In [45]:
def generate_data(data_incid=pd.DataFrame(), data_hosp=pd.DataFrame(), data_sursaud=pd.DataFrame(), data_new=pd.DataFrame(), data_vue_ensemble=pd.DataFrame(), data_metropole=pd.DataFrame(), data_vacsi=pd.DataFrame(), data_obepine=pd.DataFrame(), mode="", export_jour=False, taux_croissance=False):## Incidence
        
    dict_data = {}
    
    if export_jour:
        dict_data["jour_incid"] = list(data_incid.jour)
        dict_data["jour_hosp"] = list(data_hosp.jour)
        dict_data["jour_new"] = list(data_new.jour)
        dict_data["jour_sursaud"] = list(data_sursaud.date_de_passage)
        dict_data["jour_metropoles"] = list(data_metropole.jour.unique())
        dict_data["jour_vacsi"] = list(data_vacsi.jour)
        dict_data["jour_obepine"] = list(data_obepine.Date)
        
    if (taux_croissance) and (len(data_incid)>0):
        cas = data_incid["P"].fillna(0)
        taux_croissance_cas = ((cas-cas.shift(7))/cas.shift(7).replace(0, None)).fillna(0) * 100
        taux_croissance_cas[taux_croissance_cas>200]=200
        taux_croissance_cas[taux_croissance_cas<-200]=-200
        
        cas_rolling = data_incid["P"].rolling(window=7, center=True).mean().fillna(0)
        taux_croissance_cas_rolling = ((cas_rolling-cas_rolling.shift(7))/cas_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_cas"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_cas, 1))}
        dict_data["croissance_cas_rolling7"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_cas_rolling, 1))}
        
        tests = data_incid["T"].rolling(window=7).mean().fillna(0)
        taux_croissance_tests= ((tests-tests.shift(7))/tests.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_tests"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_tests, 1))}
        dict_data["croissance_tests_rolling7"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_tests.rolling(window=7, center=True).mean().fillna(0), 1))}
        
    if (taux_croissance) and (len(data_hosp)>0):
        hospitalisations = data_hosp.hosp.fillna(0)
        taux_croissance_hospitalisations = ((hospitalisations-hospitalisations.shift(7))/hospitalisations.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_hospitalisations"] = {"jour_nom": "jour_hosp", "valeur": list(round(taux_croissance_hospitalisations, 1))}
        dict_data["croissance_hospitalisations_rolling7"] = {"jour_nom": "jour_hosp", "valeur": list(round(taux_croissance_hospitalisations.rolling(window=7, center=True).mean().fillna(0), 1))}
        
        reanimations = data_hosp.rea.fillna(0)
        taux_croissance_reanimations = ((reanimations-reanimations.shift(7))/reanimations.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(round(taux_croissance_reanimations, 1))}
        dict_data["croissance_reanimations_rolling7"] = {"jour_nom": "jour_hosp", "valeur": list(round(taux_croissance_reanimations.rolling(window=7, center=True).mean().fillna(0), 1))}
    
    if (taux_croissance) and (len(data_new)>0):
        hospitalisations = data_new.incid_hosp.fillna(0)
        taux_croissance_hospitalisations = ((hospitalisations-hospitalisations.shift(7))/hospitalisations.shift(7).replace(0, None)).fillna(0) * 100
        
        hospitalisations_rolling = data_new.incid_hosp.fillna(0).rolling(window=7, center=True).mean()
        taux_croissance_hospitalisations_rolling = ((hospitalisations_rolling-hospitalisations_rolling.shift(7))/hospitalisations_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_incid_hospitalisations"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_hospitalisations, 1))}
        dict_data["croissance_incid_hospitalisations_rolling7"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_hospitalisations_rolling, 1))}
        
        reanimations = data_new.incid_rea.fillna(0)
        taux_croissance_reanimations = ((reanimations-reanimations.shift(7))/reanimations.shift(7).replace(0, None)).fillna(0) * 100
        
        reanimations_rolling = data_new.incid_rea.fillna(0).rolling(window=7, center=True).mean()
        taux_croissance_reanimations_rolling = ((reanimations_rolling-reanimations_rolling.shift(7))/reanimations_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_incid_reanimations"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_reanimations, 1))}
        dict_data["croissance_incid_reanimations_rolling7"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_reanimations_rolling, 1))}
    
    if(len(data_vacsi)>0):
        n_cum_dose1 = data_vacsi["n_cum_dose1"].fillna(0)
        dict_data["n_cum_dose1"] = {"jour_nom": "jour_vacsi", "valeur": list(n_cum_dose1)}
        
        n_dose1 = data_vacsi["n_dose1"].rolling(window=7).mean().fillna(0)
        dict_data["n_dose1"] = {"jour_nom": "jour_vacsi", "valeur": list(round(n_dose1, 1))}
    
    if len(data_vue_ensemble)>0:
        dict_data["jour_ehpad"] = list(data_vue_ensemble.date)
        deces_ehpad = data_vue_ensemble["total_deces_ehpad"].diff().rolling(window=7).mean().fillna(0)
        dict_data["deces_ehpad"] = {"jour_nom": "jour_ehpad", "valeur": list(round(deces_ehpad,2))}
        
        cas_spf = data_vue_ensemble.total_cas_confirmes.diff().fillna(0)
        cas_spf[cas_spf<0] = 0
        cas_spf = cas_spf.replace(to_replace=0, method='ffill')
        cas_spf_rolling = cas_spf.rolling(window=7).mean().fillna(0)
        dict_data["cas_spf"] = {"jour_nom": "jour_ehpad", "valeur": list(round(cas_spf_rolling, 2))}
        dict_data["cas_spf_brut"] = {"jour_nom": "jour_ehpad", "valeur": list(round(cas_spf, 2))}
        
    if len(data_obepine)>0:
        indicateur_obepine = data_obepine.Indicateur.fillna(0)
        
        dict_data["obepine"] = {"jour_nom": "jour_obepine", "jours":list(data_obepine.Date), "valeur": list(round(indicateur_obepine, 2))}
        
    if len(data_incid)>0:
        taux_incidence = data_incid["P"].rolling(window=7).sum().fillna(0) * 100000 / data_incid["pop"].values[0]
        dict_data["incidence"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_incidence, 1))}

        taux_positivite = (data_incid["P"] / data_incid["T"] * 100).rolling(window=7).mean().fillna(0)
        dict_data["taux_positivite"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_positivite, 2))}
        
        taux_positivite = (data_incid["P"].rolling(window=7).mean() / data_incid["T"].rolling(window=7).mean() * 100).fillna(0)
        dict_data["taux_positivite_rolling_before"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_positivite, 2))}
    
        cas = data_incid["P"].rolling(window=7).mean().fillna(0)
        dict_data["cas"] = {"jour_nom": "jour_incid", "valeur": list(round(cas, 1))}
        
        cas = data_incid["P"].fillna(0)
        dict_data["cas_brut"] = {"jour_nom": "jour_incid", "valeur": list(round(cas, 1))}
        
        cas_total = data_incid["P"].sum()
        dict_data["cas_total"] = {"jour_nom": "jour_incid", "valeur": int(cas_total)}
    
        tests = data_incid["T"].rolling(window=7).mean().fillna(0)
        dict_data["tests"] = {"jour_nom": "jour_incid", "valeur": list(round(tests, 1))}
        
        tests_total = data_incid["T"].sum()
        dict_data["tests_total"] = {"jour_nom": "jour_incid", "valeur": int(tests_total)}
        
    if (len(data_metropole)>0) & (mode=="metropoles"):
        taux_incidence = data_metropole["ti"].fillna(0)
        dict_data["incidence"] = {"jour_nom": "jour_metropoles", "valeur": list(round(taux_incidence, 1))}
        
    if (taux_croissance) and (len(data_hosp)>0):
        hospitalisations_rolling = data_hosp.hosp.rolling(window=7).mean().fillna(0)
        croissance_hospitalisations = ((hospitalisations_rolling-hospitalisations_rolling.shift(7))/hospitalisations_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_hospitalisations"] = {"jour_nom": "jour_hosp", "valeur": list(round(croissance_hospitalisations, 1))}
        
        sc_rolling = data_hosp.rea.rolling(window=7).mean().fillna(0)
        croissance_sc = ((sc_rolling-sc_rolling.shift(7))/sc_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(round(croissance_sc, 1))}
        
    if len(data_hosp)>0:
        hospitalisations = data_hosp.hosp.fillna(0)
        dict_data["hospitalisations"] = {"jour_nom": "jour_hosp", "valeur": list(hospitalisations)}

        reanimations = data_hosp.rea.fillna(0)
        dict_data["reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(reanimations)}
        
        saturation_rea = round(data_hosp["rea"]/data_hosp["LITS"].fillna(0)*100, 1)
        dict_data["saturation_reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(saturation_rea)}
    
    if len(data_new)>0:
        incid_hospitalisations = data_new.incid_hosp.rolling(window=7).mean().fillna(0)
        dict_data["incid_hospitalisations"] = {"jour_nom": "jour_new", "valeur": list(round(incid_hospitalisations, 1))}
        
        incid_hospitalisations_total = data_new.incid_hosp.sum()
        dict_data["incid_hospitalisations_total"] = {"jour_nom": "jour_new", "valeur": int(incid_hospitalisations_total)}

        incid_reanimations = data_new.incid_rea.rolling(window=7).mean().fillna(0)
        dict_data["incid_reanimations"] = {"jour_nom": "jour_new", "valeur": list(round(incid_reanimations, 1))}
        
        incid_reanimations_total = data_new.incid_rea.sum()
        dict_data["incid_reanimations_total"] = {"jour_nom": "jour_new", "valeur": int(incid_reanimations_total)}
    
    if len(data_sursaud)>0:
        nbre_acte_corona = data_sursaud.nbre_acte_corona.rolling(window=7).mean().fillna(0)
        dict_data["nbre_acte_corona"] = {"jour_nom": "jour_sursaud", "valeur": list(round(nbre_acte_corona, 1))}

        nbre_pass_corona = data_sursaud.nbre_pass_corona.rolling(window=7).mean().fillna(0)
        dict_data["nbre_pass_corona"] = {"jour_nom": "jour_sursaud", "valeur": list(round(nbre_pass_corona,  1))}
    
    if len(data_new)>0:
        deces_hospitaliers = data_new.incid_dc
        taux_croissance_deces_hospitaliers = ((deces_hospitaliers-deces_hospitaliers.shift(7))/deces_hospitaliers.shift(7).replace(0, None)).fillna(0) * 100
        
        deces_hospitaliers_rolling = data_new.incid_dc.rolling(window=7, center=False).mean().fillna(0)
        taux_croissance_deces_hospitaliers_rolling = ((deces_hospitaliers_rolling-deces_hospitaliers_rolling.shift(7))/deces_hospitaliers_rolling.shift(7).replace(0, None)).fillna(0) * 100
        
        dict_data["deces_hospitaliers"] = {"jour_nom": "jour_new", "valeur": list(round(deces_hospitaliers_rolling, 1))}
        dict_data["croissance_deces_hospitaliers"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_deces_hospitaliers, 1))}
        dict_data["croissance_deces_hospitaliers_rolling7"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_deces_hospitaliers_rolling, 1))}
        
        deces_hospitaliers_total = data_new.incid_dc.sum()
        dict_data["deces_hospitaliers_total"] = {"jour_nom": "jour_new", "valeur": int(deces_hospitaliers_total)}
    
    if len(data_incid)>0:
        population = data_incid["pop"].values[0]
        dict_data["population"] = population

    return dict_data
 

In [21]:
def generate_data_age(data_incid, data_hosp, data_adm_hosp_clage=pd.DataFrame(), export_jour=False):## Incidence
    clage_tranches = [[0], [9, 19], [29, 39], [49, 59], [69, 79], [89, 90]]
    clage_noms = ["tous", "19", "39", "59", "79", "90"]
    clage_noms_disp = ["Tous âges", "0 à 19 ans", "20 à 39 ans", "40 à 59 ans", "60 à 79 ans", "Plus de 80 ans"]
    
    dict_data = {}
    
    for (idx, clage) in enumerate(clage_tranches):
        clage_nom = clage_noms[idx]
        
        data_incid_clage = data_incid[data_incid.cl_age90.isin(clage)].groupby("jour").sum().reset_index()

        dict_data[clage_nom] = {}

        taux_incidence = data_incid_clage["P"].rolling(window=7).sum().fillna(0) * 100000 / data_incid_clage["pop"].values[0]
        dict_data[clage_nom]["incidence"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_incidence,0))}

        taux_positivite = (data_incid_clage["P"] / data_incid_clage["T"] * 100).rolling(window=7).mean().fillna(0)
        dict_data[clage_nom]["taux_positivite"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_positivite,2))}

        cas = data_incid_clage["P"].rolling(window=7).mean().fillna(0)
        dict_data[clage_nom]["cas"] = {"jour_nom": "jour_incid", "valeur": list(round(cas, 1))}
        
        taux_croissance_cas = ((cas-cas.shift(7))/cas.shift(7).replace(0, None)).fillna(0) * 100
        taux_croissance_cas[taux_croissance_cas>200] = 200
        taux_croissance_cas[taux_croissance_cas<-200] = -200
        #taux_croissance_cas = taux_croissance_cas.rolling(window=7).mean().fillna(0)
        dict_data[clage_nom]["cas_croissance_hebdo"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_cas, 1))}

        tests = data_incid_clage["T"].rolling(window=7).mean().fillna(0)
        dict_data[clage_nom]["tests"] = {"jour_nom": "jour_incid", "valeur": list(round(tests, 1))}
        
        population = data_incid_clage["pop"].values[0]
        dict_data[clage_nom]["population"] = population
        
        if (len(data_hosp)):  
            data_hosp_clage = data_hosp[data_hosp.cl_age90.isin(clage)].groupby("jour").sum().reset_index()
            hospitalisations = data_hosp_clage.hosp.fillna(0)
            dict_data[clage_nom]["hospitalisations"] = {"jour_nom": "jour_hosp", "valeur": list(hospitalisations)}

            reanimations = data_hosp_clage.rea.fillna(0)
            dict_data[clage_nom]["reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(reanimations)}

            deces_hospitaliers = data_hosp_clage.dc.diff().rolling(window=7).mean().fillna(0)
            dict_data[clage_nom]["deces_hospitaliers"] = {"jour_nom": "jour_hosp", "valeur": list(round(deces_hospitaliers, 1))}
        
        if(len(data_adm_hosp_clage)):
            df_adm_hosp_clage_temp = data_adm_hosp_clage[data_adm_hosp_clage.cl_age90.isin(clage)].groupby("jour").sum().reset_index()
            adm_hospitalisations = df_adm_hosp_clage_temp["NewAdmHospit"]
            dict_data[clage_nom]["adm_hospitalisations"] = {"jour_nom": "jour_adm_hosp_clage", "valeur": list(round(adm_hospitalisations/7, 1))}
            
    if export_jour:
            dict_data["jour_incid"] = list(data_incid.jour.unique())
            dict_data["jour_hosp"] = list(data_hosp.jour.unique())
            dict_data["tranches"] = clage_tranches
            dict_data["tranches_noms"] = clage_noms
            dict_data["tranches_noms_affichage"] = clage_noms_disp
            
            if(len(data_adm_hosp_clage)):
                dict_data["jour_adm_hosp_clage"] = list(data_adm_hosp_clage.jour.dt.strftime('%Y-%m-%d').unique())

    return dict_data
 

In [64]:
def generate_data_niveaux_scolaires(data_incid_niveaux_scolaires, export_jour=False):
    dict_data={}
    clage_tranches = ["0", "2", "5", "10", "14", "17", "18"]
    clage_noms = ["tous_scol", "02_scol", "05_scol", "10_scol", "14_scol", "17_scol", "18_scol"]
    clage_noms_disp = ["Tous", "0 - 2 ans", "3 - 5 ans", "6 - 10 ans", "11 - 14 ans", "15 - 17 ans", "plus 18 ans"]
    
    for (idx, clage) in enumerate(clage_tranches):
        clage_nom = clage_noms[idx]
        data_incid_niveaux_scolaires_clage = data_incid_niveaux_scolaires[data_incid_niveaux_scolaires.age_18ans == clage].reset_index()

        dict_data[clage_nom] = {}

        taux_incidence = data_incid_niveaux_scolaires_clage["ti"]
        taux_depistage = data_incid_niveaux_scolaires_clage["td"]
        taux_positivite = data_incid_niveaux_scolaires_clage["tp"]
        
        dict_data[clage_nom]["incidence"] = {"jour_nom": "jour_niveaux_scolaires", "valeur": list(np.round(taux_incidence.values))}
        dict_data[clage_nom]["depistage"] = {"jour_nom": "jour_niveaux_scolaires", "valeur": list(np.round(taux_depistage.values))}
        dict_data[clage_nom]["positivite"] = {"jour_nom": "jour_niveaux_scolaires", "valeur": list(np.round(taux_positivite.values))}
            
    if export_jour:
            dict_data["jour_niveaux_scolaires"] = list(data_incid_niveaux_scolaires.jour.unique().astype(str))
            dict_data["tranches_scolaires"] = clage_tranches
            dict_data["tranches_noms_scolaires"] = clage_noms
            dict_data["tranches_noms_affichage_scolaires"] = clage_noms_disp

    return dict_data
 

In [25]:
def generate_data_education(df_education):
    clage_noms = [2, 5, 10, 14, 17, 18]
    clage_disp = ["0 - 2 ans", "3 - 5 ans", "6 - 10 ans", "11 - 14 ans", "15 - 17 ans", "Plus 18 ans"]
    
    dict_data = {}
    dict_data["tranches_noms"] = clage_noms
    dict_data["tranches_noms_affichage"] = clage_disp
    
    for clage in clage_noms:
        dict_data[clage] = {}
        
        df_temp = df_education[df_education["age_18ans"]==clage]
        
        dict_data[clage]["ti"] = df_temp["ti"].fillna(0).tolist()
        dict_data[clage]["tp"] = df_temp["tp"].fillna(0).tolist()
        dict_data[clage]["td"] = df_temp["td"].fillna(0).tolist()
        
    return dict_data

In [22]:
def export_data(data, suffix=""):
    with open(PATH_STATS + 'dataexplorer{}.json'.format(suffix), 'w') as outfile:
        json.dump(data, outfile)

In [27]:
def dataexplorer():
    dict_data = {}
    
    dict_data["regions"] = sorted(regions)
    dict_data["metropoles"] = sorted(metropoles)
    dict_data["departements"] = departements
    dict_data["france"] = generate_data(df_incid_fra, df_france, df_sursaud_france, df_new_france, df_vue_ensemble, data_metropole=df_metro_0, data_vacsi=df_vacsi, data_obepine=df_obepine_france, mode="france", export_jour=True, taux_croissance=True)
    dict_data["metropole"] = generate_data(df_incid[df_incid["dep"].str.len()<=2].groupby(["jour"]).sum(), df[df["dep"].str.len()<=2].groupby(["jour"]).sum(), df_sursaud[df_sursaud["dep"].str.len()<=2].groupby(["date_de_passage"]).sum(), df_new[df_new["dep"].str.len()<=2].groupby(["jour"]).sum(), data_vacsi=df_vacsi_dep[df_new["dep"].str.len()<=2].groupby(["jour"]).sum())
    dict_data["drom_com"] = generate_data(df_incid[df_incid["dep"].str.len()>2].groupby(["jour"]).sum(), df[df["dep"].str.len()>2].groupby(["jour"]).sum(), df_sursaud[df_sursaud["dep"].str.len()>2].groupby(["date_de_passage"]).sum(), df_new[df_new["dep"].str.len()>2].groupby(["jour"]).sum(), data_vacsi=df_vacsi_dep[df_new["dep"].str.len()>2].groupby(["jour"]).sum())

    noms_departements={}
    
    for reg in regions:
        
        dict_data[reg] = generate_data(df_incid_regions[df_incid_regions.regionName==reg], \
                                       df_regions[df_regions.regionName==reg],\
                                       df_sursaud_regions[df_sursaud_regions.regionName==reg],
                                       df_new_regions[df_new_regions.regionName==reg],
                                       data_vacsi=df_vacsi_reg[df_vacsi_reg.regionName==reg],\
                                       data_obepine=df_obepine[df_obepine.regionName==reg]
                                      )
        #print(df_vacsi_reg[df_vacsi_reg.regionName==reg])
    
    for dep in departements:
        df_incid_dep = df_incid[df_incid.dep==dep]
        dict_data[dep] = generate_data(df_incid_dep, df[df.dep==dep], df_sursaud[df_sursaud.dep==dep], df_new[df_new.dep==dep], data_vacsi=df_vacsi_dep[df_vacsi_dep.dep==dep])
        
        noms_departements[dep] = df_incid_dep["departmentName"].values[0]
    dict_data["departements_noms"] = noms_departements
    
    for zone in [zone_a, zone_b, zone_c]:
        df_incid_zone = df_incid[df_incid.dep.isin(zone)].groupby("jour").sum().reset_index()
        df_zone = df[df.dep.isin(zone)].groupby("jour").sum().reset_index()
        df_sursaud_zone = df_sursaud[df_sursaud.dep.isin(zone)].groupby("date_de_passage").sum().reset_index()
        df_new_zone = df_new[df_new.dep.isin(zone)].groupby("jour").sum().reset_index()
        df_vacsi_zone = df_vacsi_dep[df_vacsi_dep.dep.isin(zone)].groupby("jour").sum().reset_index()
        
        dict_data[zone[0]] = generate_data(df_incid_zone, df_zone, df_sursaud_zone, df_new_zone, data_vacsi=df_vacsi_zone)
    
    # Confinés mars 2021
    df_incid_zone = df_incid[df_incid.dep.isin(confines_mars_2021)].groupby("jour").sum().reset_index()
    df_zone = df[df.dep.isin(confines_mars_2021)].groupby("jour").sum().reset_index()
    df_sursaud_zone = df_sursaud[df_sursaud.dep.isin(confines_mars_2021)].groupby("date_de_passage").sum().reset_index()
    df_new_zone = df_new[df_new.dep.isin(confines_mars_2021)].groupby("jour").sum().reset_index()
    df_vacsi_zone = df_vacsi_dep[df_vacsi_dep.dep.isin(confines_mars_2021)].groupby("jour").sum().reset_index()
    
    dict_data["confines_mars_2021"] = generate_data(df_incid_zone, df_zone, df_sursaud_zone, df_new_zone, data_vacsi=df_vacsi_zone)
        
    for metropole in metropoles:
        print(metropole)
        dict_data[metropole] = generate_data(data_metropole=df_metro_0[df_metro_0.Metropole == metropole], mode="metropoles")
        #print(dict_data[metropole])
        
    dict_data["zones_vacances"] = ["zone_a", "zone_b", "zone_c"]
    
    export_data(dict_data, suffix="_compr")

In [28]:
def dataexplorer_age():
    dict_data = {}
    regions_tests_viros = list(dict.fromkeys(list(df_tests_viros_enrichi['regionName'].dropna().values))) 
    departements_tests_viros = list(dict.fromkeys(list(df_tests_viros_enrichi['dep'].dropna().values))) 
    dict_data["regions"] = sorted(regions_tests_viros)
    dict_data["departements"] = sorted(departements_tests_viros)
    
    dict_data["france"] = generate_data_age(df_tests_viros_france, df_hosp_clage_france, data_adm_hosp_clage=df_adm_hosp_clage_france, export_jour=True)
    
    for reg in regions_tests_viros:
        dict_data[reg] = generate_data_age(df_tests_viros_regions[df_tests_viros_regions.regionName == reg],\
                                           df_hosp_clage_regions[df_hosp_clage_regions.regionName == reg]) #data_adm_hosp_clage=data_adm_hosp_clage
    noms_departements={}
    for dep in departements_tests_viros:
        df_tests_viros_enrichi_temp = df_tests_viros_enrichi[df_tests_viros_enrichi.dep == dep]
        dict_data[dep] = generate_data_age(df_tests_viros_enrichi_temp,\
                                           pd.DataFrame())
        
        nom_dep = df_tests_viros_enrichi_temp["departmentName"].values[0]
        
        if(type(nom_dep) is float): #Pas de nom, nom_dep == NaN
            #print(dep)
            nom_dep = "--"
        
        noms_departements[dep] = nom_dep
        
    dict_data["departements_noms"] = noms_departements
    
    export_data(dict_data, suffix="_compr_age")
    return dict_data

In [29]:
def dataexplorer_education():
    dict_data = {}
    dict_data["france"] = generate_data_education(df_education=df_education)
    export_data(dict_data, suffix="_education")

In [65]:
def dataexplorer_niveaux_scolaires():
    df_niveaux_scolaires_dep_sorted = df_niveaux_scolaires_dep.sort_values(by="dep")
    df_niveaux_scolaires_reg_sorted = df_niveaux_scolaires_reg.sort_values(by="reg")
    
    dict_data = {}
    
    dict_data["departements_noms"] = list(df_niveaux_scolaires_dep_sorted["departmentName"].unique().astype(str))
    dict_data["departements"] = list(df_niveaux_scolaires_dep_sorted["dep"].unique().astype(str))
    
    dict_data["regions_noms"] = list(df_niveaux_scolaires_reg_sorted["regionName"].unique().astype(str))
    dict_data["regions"] = list(df_niveaux_scolaires_reg_sorted["reg"].unique().astype(str))
    
    dict_data["france"] = generate_data_niveaux_scolaires(data_incid_niveaux_scolaires=df_niveaux_scolaires_fra, export_jour=True)
    
    for dep in dict_data["departements"]:
        data_incid_niveaux_scolaires_dep_temp = df_niveaux_scolaires_dep[df_niveaux_scolaires_dep["dep"] == dep]
        dict_data[dep] = generate_data_niveaux_scolaires(data_incid_niveaux_scolaires=data_incid_niveaux_scolaires_dep_temp, export_jour=False)
    
    export_data(dict_data, suffix="_education")
    return dict_data

In [ ]:
df_ns = dataexplorer_niveaux_scolaires()

In [66]:
df_ns["01"]["jour_niveaux_scolaires"]

['2020-05-19T00:00:00.000000000',
 '2020-05-20T00:00:00.000000000',
 '2020-05-21T00:00:00.000000000',
 '2020-05-22T00:00:00.000000000',
 '2020-05-23T00:00:00.000000000',
 '2020-05-24T00:00:00.000000000',
 '2020-05-25T00:00:00.000000000',
 '2020-05-26T00:00:00.000000000',
 '2020-05-27T00:00:00.000000000',
 '2020-05-28T00:00:00.000000000',
 '2020-05-29T00:00:00.000000000',
 '2020-05-30T00:00:00.000000000',
 '2020-05-31T00:00:00.000000000',
 '2020-06-01T00:00:00.000000000',
 '2020-06-02T00:00:00.000000000',
 '2020-06-03T00:00:00.000000000',
 '2020-06-04T00:00:00.000000000',
 '2020-06-05T00:00:00.000000000',
 '2020-06-06T00:00:00.000000000',
 '2020-06-07T00:00:00.000000000',
 '2020-06-08T00:00:00.000000000',
 '2020-06-09T00:00:00.000000000',
 '2020-06-10T00:00:00.000000000',
 '2020-06-11T00:00:00.000000000',
 '2020-06-12T00:00:00.000000000',
 '2020-06-13T00:00:00.000000000',
 '2020-06-14T00:00:00.000000000',
 '2020-06-15T00:00:00.000000000',
 '2020-06-16T00:00:00.000000000',
 '2020-06-17T0

In [78]:
dataexplorer()

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


Nice
Marseille
Dijon
Brest
Toulouse
Bordeaux
Montpellier
Rennes
Tours
Grenoble
Saint Etienne
Nantes
Orléans
Nancy
Metz
Lille
Clermont-Auvergne
Strasbourg
Lyon
Paris
Rouen
Toulon


In [33]:
dict_data = dataexplorer_age()

In [34]:
dataexplorer_education()